In [19]:
#### LIBRERIE ###
import pandas as pd
from datetime import datetime
from bs4 import BeautifulSoup
import requests
import re
import random
import time

In [20]:
def estrazione_codici_delle_codici_squadre(url):#estrazione dei codici univoci delle squadre
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        # Trova tutti i tag <a> con l'attributo href che contiene "/squadre/"
        squad_links = soup.find_all('a', href=lambda x: x and "/squadre/" in x)

        # Estrai i codici dalle URL
        codici_squadre = {}
        for link in squad_links:
            # Ottieni il codice dalla parte finale dell'URL
            url_parts = link['href'].split('/')
            codice_squadra = url_parts[-2]

            # Usa il nome della squadra come chiave e il codice come valore nel dizionario
            codici_squadre[link.get_text()] = codice_squadra

        return codici_squadre
    else:
        print(f"Errore nella richiesta HTTP per {url}: {response.status_code}")
        return {}
def dizionario_codici():
    url_seriea = 'https://fbref.com/it/comp/11/Statistiche-di-Serie-A'
    codici_squadre_seriea = estrazione_codici_delle_codici_squadre(url_seriea)
    codici_squadre_seriea = dict(list(codici_squadre_seriea.items())[:21])
    squadre_disponibili_seriea = list(codici_squadre_seriea.keys())


    url_premier = 'https://fbref.com/it/comp/9/Statistiche-di-Premier-League'
    codici_squadre_premier = estrazione_codici_delle_codici_squadre(url_premier)
    codici_squadre_premier = dict(list(codici_squadre_premier.items())[:21])
    squadre_disponibili_premier = list(codici_squadre_premier.keys())


    url_laliga = 'https://fbref.com/it/comp/12/Statistiche-di-La-Liga'
    codici_squadre_laliga = estrazione_codici_delle_codici_squadre(url_laliga)
    codici_squadre_laliga = dict(list(codici_squadre_laliga.items())[:21])
    squadre_disponibili_laliga = list(codici_squadre_laliga.keys())

    url_ligueone = 'https://fbref.com/it/comp/13/Statistiche-di-Ligue-1'
    codici_squadre_ligueone= estrazione_codici_delle_codici_squadre(url_ligueone)
    codici_squadre_ligueone = dict(list(codici_squadre_ligueone.items())[:21])
    squadre_disponibili_ligueone = list(codici_squadre_ligueone.keys())

    url_bunds = 'https://fbref.com/it/comp/20/Statistiche-di-Bundesliga'
    codici_squadre_bunds= estrazione_codici_delle_codici_squadre(url_bunds)
    codici_squadre_bunds = dict(list(codici_squadre_bunds.items())[:21])
    squadre_disponibili_bunds = list(codici_squadre_bunds.keys())
    

    url_champions = 'https://fbref.com/it/comp/8/Statistiche-di-Champions-League'
    codici_squadre_champions= estrazione_codici_delle_codici_squadre(url_champions)
    codici_squadre_champions = dict(list(codici_squadre_champions.items())[:21])
    squadre_disponibili_champions = list(codici_squadre_champions.keys())


    url_europa = 'https://fbref.com/it/comp/19/Statistiche-di-Europa-League'
    codici_squadre_europa= estrazione_codici_delle_codici_squadre(url_europa)
    codici_squadre_europa = dict(list(codici_squadre_europa.items())[:21])
    squadre_disponibili_europa = list(codici_squadre_europa.keys())

    # Dizionario per i campionati nazionali
    campionati = {
        "Serie A": codici_squadre_seriea,
        "Premier League": codici_squadre_premier,
        "La Liga": codici_squadre_laliga,
        "Ligue 1": codici_squadre_ligueone,
        "Bundesliga": codici_squadre_bunds
    }

    # Dizionario per le coppe internazionali
    coppe = {
        "Champions League": codici_squadre_champions,
        "Europa League": codici_squadre_europa
    }

    return campionati, coppe
campionati, coppe = dizionario_codici()
# Accedere al dizionario "campionati" all'interno del dizionario principale
campionati_seriea = campionati["Serie A"]

In [56]:
def create_links(base_url, squadre_dict, squadra, comp_types):
    codice_squadra = squadre_dict.get(squadra)
    if codice_squadra:
        links = [f"{base_url}/{codice_squadra}/2023-2024/partite/all_comps/{comp_type}/" for comp_type in comp_types]
        # Aggiungiamo anche i link per il 2022-2023 e il 2021-2022
        links2 = [f"{base_url}/{codice_squadra}/2022-2023/partite/all_comps/{comp_type}/" for comp_type in comp_types]
        links3 = [f"{base_url}/{codice_squadra}/2021-2022/partite/all_comps/{comp_type}/" for comp_type in comp_types]
        links4 = [f"{base_url}/{codice_squadra}/2023-2024/goallogs/all_comps"]
        links5 = [f"{base_url}/{codice_squadra}/2022-2023/goallogs/all_comps"]
        links6 = [f"{base_url}/{codice_squadra}/2021-2022/goallogs/all_comps"]
        return links, links2, links3, links4, links5, links6
    else:
        return []
base_url = "https://fbref.com/it/squadre"
comp_types = ['schedule','shooting', 'keeper', 'passing', 'passing_types', 'gca', 'defense', 'possession', 'misc']
links, links2, links3, links4, links5, links6 = create_links(base_url, campionati_seriea, 'Lazio', comp_types)

In [59]:
##### SCARICAMENTO E PULIZIA DATI REGISTRO DEI GOL
def download_data(links):
    all_dataframes = []
    for link in links:
        try:
            ritardo = random.uniform(0.88, 1.20)
            time.sleep(ritardo)
            dataframe = pd.read_html(link)[0]  # Assume che la tabella che ti interessa sia la prima
            all_dataframes.append(dataframe)
            print(f"Dati scaricati con successo da {link}")
        except Exception as e:
            print(f"Errore durante il download dei dati da {link}: {e}")
    concatenated_df = pd.concat(all_dataframes, ignore_index=True)
    return concatenated_df
# Scarica i dati da ciascun link
dataframes4 = download_data(links4)
dataframes5 = download_data(links5)
dataframes6 = download_data(links6)

Dati scaricati con successo da https://fbref.com/it/squadre/7213da33/2023-2024/goallogs/all_comps
Dati scaricati con successo da https://fbref.com/it/squadre/7213da33/2022-2023/goallogs/all_comps
Dati scaricati con successo da https://fbref.com/it/squadre/7213da33/2021-2022/goallogs/all_comps


In [48]:
concatenated_df = pd.concat(dataframes4)
# Salva il DataFrame concatenato in un file CSV
concatenated_df.to_csv('output.csv', index=False)
print(concatenated_df)

    Pos.        Data Competizione          Girone      Stadio  \
0      1  2023-08-20      Serie A   Giornata n. 1     In casa   
1      2  2023-08-20      Serie A   Giornata n. 1     In casa   
2      3  2023-08-26      Serie A   Giornata n. 2  Fuori casa   
3      4  2023-09-01      Serie A   Giornata n. 3     In casa   
4      5  2023-09-17      Serie A   Giornata n. 4     In casa   
..   ...         ...          ...             ...         ...   
79    80  2024-04-22      Serie A  Giornata n. 33     In casa   
80    81  2024-04-25      Serie A  Giornata n. 32  Fuori casa   
81    82  2024-04-25      Serie A  Giornata n. 32  Fuori casa   
82    83  2024-04-28      Serie A  Giornata n. 34  Fuori casa   
83    84  2024-04-28      Serie A  Giornata n. 34  Fuori casa   

              Marcatore     Avversario F Inz    xG  PSxG  ... Distance  \
0        Andrea Belotti    Salernitana    Sì  0.49  0.30  ...     11.0   
1        Andrea Belotti    Salernitana    Sì  0.41  0.42  ...      6.0 

In [60]:
def download_data(links):
    all_dataframes = []
    for link in links:
        try:
            ritardo = random.uniform(0.88, 1.20)
            time.sleep(ritardo)
            dataframe = pd.read_html(link)[0]  # Assume che la tabella che ti interessa sia la prima
            all_dataframes.append(dataframe)
            print(f"Dati scaricati con successo da {link}")
        except Exception as e:
            print(f"Errore durante il download dei dati da {link}: {e}")
            
    return all_dataframes
# Scarica i dati da ciascun link
dataframes = download_data(links)
dataframes2 = download_data(links2)
dataframes3 = download_data(links3)


Dati scaricati con successo da https://fbref.com/it/squadre/7213da33/2023-2024/partite/all_comps/schedule/
Dati scaricati con successo da https://fbref.com/it/squadre/7213da33/2023-2024/partite/all_comps/shooting/
Dati scaricati con successo da https://fbref.com/it/squadre/7213da33/2023-2024/partite/all_comps/keeper/
Dati scaricati con successo da https://fbref.com/it/squadre/7213da33/2023-2024/partite/all_comps/passing/
Dati scaricati con successo da https://fbref.com/it/squadre/7213da33/2023-2024/partite/all_comps/passing_types/
Dati scaricati con successo da https://fbref.com/it/squadre/7213da33/2023-2024/partite/all_comps/gca/
Dati scaricati con successo da https://fbref.com/it/squadre/7213da33/2023-2024/partite/all_comps/defense/
Dati scaricati con successo da https://fbref.com/it/squadre/7213da33/2023-2024/partite/all_comps/possession/
Dati scaricati con successo da https://fbref.com/it/squadre/7213da33/2023-2024/partite/all_comps/misc/
Dati scaricati con successo da https://fbre

In [35]:
##### SCARICAMENTO E PULIZIA DATI REGISTRO PARTITE
def download_data(links):
    all_dataframes = []
    for link in links:
        try:
            ritardo = random.uniform(0.88, 1.20)
            time.sleep(ritardo)
            dataframe = pd.read_html(link)[0]  # Assume che la tabella che ti interessa sia la prima
            all_dataframes.append(dataframe)
            print(f"Dati scaricati con successo da {link}")
        except Exception as e:
            print(f"Errore durante il download dei dati da {link}: {e}")
            
    return all_dataframes
# Scarica i dati da ciascun link
dataframes = download_data(links)
dataframes2 = download_data(links2)
dataframes3 = download_data(links3)
def mange_merge(dataframes):
    zero = dataframes[0]
    # Lista di indici delle colonne da mantenere
    fd_indici_casa = [0, 1, 2, 3, 5, 6, 7, 8, 12, 13]  # Sostituisci con gli indici delle colonne che vuoi mantenere
    # Filtra il dataset mantenendo solo le colonne specificate
    zero = zero.iloc[:, fd_indici_casa]
    # Converte la colonna 'Data' in formato datetime
    zero.loc[:, 'Data'] = pd.to_datetime(zero['Data'], format='%d-%m-%Y')

    # Filtra le righe in base alla data odierna
    data_odierna = datetime.now()
    zero = zero[zero['Data'] <= data_odierna]

    uno = dataframes[1]
    uno = uno.reset_index(drop=True, level=uno.index.names)
    uno.columns = uno.columns.droplevel(level=0)
    # Nome della colonna da dividere
    colonna_da_dividere = '% TiP'  # Sostituisci con il nome effettivo della tua colonna

    # Dividi i valori della colonna per 10
    uno[colonna_da_dividere] = uno[colonna_da_dividere] / 100
    # Lista di indici delle colonne da mantenere
    fd_indici_casa2 = [13, 14, 15, 16, 17, 18, 19]  # Sostituisci con gli indici delle colonne che vuoi mantenere
    # Filtra il dataset mantenendo solo le colonne specificate
    uno = uno.iloc[:-1, fd_indici_casa2]
    due = dataframes[2]
    due = due.reset_index(drop=True, level=due.index.names)
    due.columns = due.columns.droplevel(level=0)
    # Nome della colonna da dividere
    colonna_da_dividere2 = '%Parate'  # Sostituisci con il nome effettivo della tua colonna
    # Nome della colonna da dividere
    colonna_da_dividere3 = '% compl.'  # Sostituisci con il nome effettivo della tua colonna
    # Nome della colonna da dividere
    colonna_da_dividere4 = '% lanci'  # Sostituisci con il nome effettivo della tua colonna
    # Nome della colonna da dividere
    colonna_da_dividere5 = '% lanci'  # Sostituisci con il nome effettivo della tua colonna
    # Nome della colonna da dividere
    colonna_da_dividere6 = '% par.'  # Sostituisci con il nome effettivo della tua colonna

    # Dividi i valori della colonna per 10
    due[colonna_da_dividere2] = due[colonna_da_dividere2] / 10
    # Dividi i valori della colonna per 10
    due[colonna_da_dividere3] = due[colonna_da_dividere3] / 10
    # Dividi i valori della colonna per 10
    due[colonna_da_dividere4] = due[colonna_da_dividere4] / 10
    # Dividi i valori della colonna per 10
    due[colonna_da_dividere5] = due[colonna_da_dividere5] / 10
    # Dividi i valori della colonna per 10
    due[colonna_da_dividere6] = due[colonna_da_dividere6] / 10

    # Lista di indici delle colonne da mantenere
    fd_indici_casa3 = [13, 14, 23, 26, 29, 33, 34] # Sostituisci con gli indici delle colonne che vuoi mantenere
    # Filtra il dataset mantenendo solo le colonne specificate
    due = due.iloc[:-1, fd_indici_casa3]
    tre = dataframes[3]
    tre = tre.reset_index(drop=True, level=tre.index.names)
    tre.columns = tre.columns.droplevel(level=0)
    # Lista di indici delle colonne da mantenere
    colonna_da_dividere7 = '% compl.'  # Sostituisci con il nome effettivo della tua colonna

    tre[colonna_da_dividere7] = tre[colonna_da_dividere7] / 10

    fd_indici_casa4 = [12, 17, 20, 23, 27, 28, 29, 31] # Sostituisci con gli indici delle colonne che vuoi mantenere
    # Filtra il dataset mantenendo solo le colonne specificate
    tre = tre.iloc[:-1, fd_indici_casa4]
    quattro = dataframes[4]
    quattro = quattro.reset_index(drop=True, level=quattro.index.names)
    quattro.columns = quattro.columns.droplevel(level=0)
    # Lista di indici delle colonne da mantenere
    fd_indici_casa5 = [11, 13, 14, 15, 22, 23, 24] # Sostituisci con gli indici delle colonne che vuoi mantenere
    # Filtra il dataset mantenendo solo le colonne specificate
    quattro = quattro.iloc[:-1, fd_indici_casa5]
    cinque = dataframes[5]
    cinque = cinque.reset_index(drop=True, level=cinque.index.names)
    cinque.columns = cinque.columns.droplevel(level=0)
    # Lista di indici delle colonne da mantenere
    fd_indici_casa6 = [10,17] # Sostituisci con gli indici delle colonne che vuoi mantenere
    # Filtra il dataset mantenendo solo le colonne specificate
    cinque = cinque.iloc[:-1, fd_indici_casa6]
    sei = dataframes[6]
    sei = sei.reset_index(drop=True, level=sei.index.names)
    sei.columns = sei.columns.droplevel(level=0)
    # Lista di indici delle colonne da mantenere
    colonna_da_dividere8 = '% cntrs'  # Sostituisci con il nome effettivo della tua colonna

    sei[colonna_da_dividere8] = sei[colonna_da_dividere8] / 10
    # Lista di indici delle colonne da mantenere
    fd_indici_casa7 = [11, 17, 22, 24, 25] # Sostituisci con gli indici delle colonne che vuoi mantenere
    # Filtra il dataset mantenendo solo le colonne specificate
    sei = sei.iloc[:-1, fd_indici_casa7]
    sette = dataframes[7]
    sette = sette.reset_index(drop=True, level=sette.index.names)
    sette.columns = sette.columns.droplevel(level=0)
    colonna_da_dividere10 = 'Succ%'  # Sostituisci con il nome effettivo della tua colonna
    colonna_da_dividere11 = 'Tkld%'
    sette[colonna_da_dividere10] = sette[colonna_da_dividere10] / 10
    sette[colonna_da_dividere11] = sette[colonna_da_dividere11] / 10
    # Lista di indici delle colonne da mantenere
    fd_indici_casa8 = [16, 20, 22, 26, 27, 30, 31, 32] # Sostituisci con gli indici delle colonne che vuoi mantenere
    # Filtra il dataset mantenendo solo le colonne specificate
    sette = sette.iloc[:-1, fd_indici_casa8]
    otto = dataframes[8]
    otto = otto.reset_index(drop=True, level=otto.index.names)
    otto.columns = otto.columns.droplevel(level=0)
    colonna_da_dividere9 = '% vinti'  # Sostituisci con il nome effettivo della tua colonna

    otto[colonna_da_dividere9] = otto[colonna_da_dividere9] / 10
    # Lista di indici delle colonne da mantenere
    fd_indici_casa9 = [10,11,12, 13, 14, 15, 18, 21, 22, 25] # Sostituisci con gli indici delle colonne che vuoi mantenere
    # Filtra il dataset mantenendo solo le colonne specificate
    otto = otto.iloc[:-1, fd_indici_casa9]

    
    data_merge = pd.concat([zero, uno, due, tre, quattro, cinque, sei, sette, otto], axis=1)
    data_merge = data_merge.fillna(0)
    #data_merge3 = data_merge2[data_merge2['Competizione'] == 'Serie A', 'Champions Lg', 'Premier League']

    # Mappa delle sostituzioni
    mappa_sostituzioni = {'V': 3, 'N': 1, 'P': 0}

    # Sostituisci i valori nella colonna "risultato" utilizzando la mappa
    data_merge['Risultato'] = data_merge['Risultato'].replace(mappa_sostituzioni)

    return data_merge
zero = mange_merge(dataframes)
uno = mange_merge(dataframes2)
due = mange_merge(dataframes3)
last3 = pd.concat([due, uno,zero], ignore_index=True)
def modifica_valori(dataframe):
    # Modifica i valori all'interno delle colonne 'Rf' e 'Rs'
    dataframe['Rf'] = dataframe['Rf'].apply(lambda x: int(x.split()[0]) if isinstance(x, str) else x)
    dataframe['Rs'] = dataframe['Rs'].apply(lambda x: int(x.split()[0]) if isinstance(x, str) else x)
    return dataframe
k=modifica_valori(last3)
k
#creare pulizia dei registri gol 

,Data,Ora,Competizione,Girone,Stadio,Risultato,Rf,Rs,Poss.,Spettatori,...,Amm.,Esp.,2 CG,Falli,Falli.1,Fuorigioco,Contr. vinti,AR,Recup.,% vinti
0,2021-08-21,18:30,Serie A,Giornata n. 1,Casa,3,4,0,60.0,27.402,...,1.0,0.0,0.0,18.0,6.0,2.0,10.0,0.0,43.0,50.0
1,2021-08-27,20:45,Serie A,Giornata n. 2,Ospiti,3,3,1,57.0,11.214,...,2.0,0.0,0.0,12.0,13.0,0.0,13.0,0.0,62.0,50.0
2,2021-09-12,12:30,Serie A,Giornata n. 3,Ospiti,1,2,2,48.0,7.441,...,4.0,0.0,0.0,13.0,12.0,1.0,7.0,0.0,48.0,56.5
3,2021-09-15,21:00,Champions Lg,Fase a gironi,Casa,0,0,1,47.0,37.082,...,1.0,0.0,0.0,9.0,10.0,4.0,8.0,0.0,53.0,69.2
4,2021-09-18,18:00,Serie A,Giornata n. 4,Casa,3,6,1,45.0,35.278,...,0.0,0.0,0.0,10.0,10.0,2.0,8.0,0.0,41.0,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,2024-04-08,20:45,Serie A,Giornata n. 31,Ospiti,3,2,1,76.0,24.993,...,2.0,0.0,0.0,11.0,11.0,4.0,8.0,0.0,41.0,40.0
151,2024-04-14,20:45,Serie A,Giornata n. 32,Casa,1,2,2,63.0,72.559,...,0.0,0.0,0.0,5.0,9.0,3.0,4.0,0.0,25.0,50.0
152,2024-04-22,20:45,Serie A,Giornata n. 33,Ospiti,3,2,1,48.0,75.554,...,2.0,1.0,0.0,10.0,19.0,5.0,9.0,0.0,41.0,56.3
153,2024-04-28,12:30,Serie A,Giornata n. 34,Casa,3,2,0,62.0,0.000,...,0.0,0.0,0.0,6.0,7.0,2.0,6.0,0.0,39.0,64.7


In [33]:
###FUNZIONI    
def filtro_giornata_odierna(dataset, colonne_selezionate = ["Data","Ora", "Casa", "Ospiti", "Competizione"]):
        # Ottenere la data odierna
        oggi = datetime.today().strftime('%d-%m-%Y')

        # Filtrare il dataset in base alla giornata odierna
        dataset_odierno = dataset[dataset["Data"] == oggi][colonne_selezionate]

        return dataset_odierno
#creazione del calendario top campionati 
def partite_del_giorno():
    colonne_selezionate = ["Data","Ora", "Casa", "Ospiti"]
    calendar = pd.read_html("https://fbref.com/it/comp/11/calendario/Risultati-e-partite-di-Serie-A")
    calendar_seriea = calendar[0]
    calendar_seriea['Casa']
    calendar_seriea['Ospiti']
    seriea = calendar_seriea[colonne_selezionate]
    seriea['Competizione'] = "seriea"


    calendar1 = pd.read_html("https://fbref.com/it/comp/9/calendario/Risultati-e-partite-di-Premier-League")
    calendar_premier = calendar1[0]
    calendar_premier['Casa']
    calendar_premier['Ospiti']
    premier = calendar_premier[colonne_selezionate]
    premier['Competizione'] = "premierleague"

    calendar_2 = pd.read_html("https://fbref.com/it/comp/12/calendario/Risultati-e-partite-di-La-Liga")
    calendar_liga = calendar_2[0]
    calendar_liga['Casa']
    calendar_liga['Ospiti']
    liga = calendar_liga[colonne_selezionate]
    liga['Competizione'] = "laliga"

    calendar_3 = pd.read_html("https://fbref.com/it/comp/13/calendario/Risultati-e-partite-di-Ligue-1")
    calendar_ligue = calendar_3[0]
    calendar_ligue['Casa']
    calendar_ligue['Ospiti']
    ligue = calendar_ligue[colonne_selezionate]
    ligue['Competizione'] = "ligueone"

    calendar_4 = pd.read_html("https://fbref.com/it/comp/20/calendario/Risultati-e-partite-di-Bundesliga")
    calendar_bundes = calendar_4[0]
    calendar_bundes['Casa']
    calendar_bundes['Ospiti']
    bundes = calendar_bundes[colonne_selezionate]
    bundes['Competizione'] = "bundesliga"

    calendar_5 = pd.read_html("https://fbref.com/it/comp/8/calendario/Risultati-e-partite-di-Champions-League")
    calendar_champions = calendar_5[0]
    calendar_champions['Casa']
    calendar_champions['Ospiti']
    champions = calendar_champions[colonne_selezionate]
    champions['Competizione'] = "champions"
            
    return seriea, premier, liga, ligue, bundes, champions
def unisci_dataset(lista_dataset):
    # Verifica che la lista dei dataset non sia vuota
    if len(lista_dataset) == 0:
        raise ValueError("La lista dei dataset è vuota.")
    
    # Unisce i dataset
    merged_dataset = pd.concat(lista_dataset, ignore_index=True)
    
    return merged_dataset
seriea, premier, liga, ligue, bundes, champions = partite_del_giorno()

print(seriea)
filtro_giornata_odierna(unisci_dataset([seriea, premier, liga, ligue, bundes]))

<ipython-input-33-c8dbb346659b>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seriea['Competizione'] = "seriea"
<ipython-input-33-c8dbb346659b>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  premier['Competizione'] = "premierleague"
<ipython-input-33-c8dbb346659b>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user

           Data    Ora           Casa         Ospiti Competizione
0    19-08-2023  18:30      Frosinone         Napoli       seriea
1    19-08-2023  18:30         Empoli  Hellas Verona       seriea
2    19-08-2023  20:45          Inter          Monza       seriea
3    19-08-2023  20:45          Genoa     Fiorentina       seriea
4    20-08-2023  18:30       Sassuolo       Atalanta       seriea
..          ...    ...            ...            ...          ...
416  26-05-2024    NaN          Milan    Salernitana       seriea
417  26-05-2024    NaN       Atalanta         Torino       seriea
418  26-05-2024    NaN       Juventus          Monza       seriea
419  26-05-2024    NaN         Napoli          Lecce       seriea
420  26-05-2024    NaN  Hellas Verona          Inter       seriea

[421 rows x 5 columns]


<ipython-input-33-c8dbb346659b>:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  champions['Competizione'] = "champions"


,Data,Ora,Casa,Ospiti,Competizione
381,05-05-2024,12:30,Cagliari,Lecce,seriea
382,05-05-2024,15:00,Empoli,Frosinone,seriea
383,05-05-2024,15:00,Hellas Verona,Fiorentina,seriea
384,05-05-2024,18:00,Milan,Genoa,seriea
385,05-05-2024,20:45,Roma,Juventus,seriea
814,05-05-2024,14:00,Brighton,Aston Villa,premierleague
815,05-05-2024,14:00,Chelsea,West Ham,premierleague
816,05-05-2024,16:30,Liverpool,Tottenham,premierleague
1216,05-05-2024,14:00,Osasuna,Betis,laliga
1217,05-05-2024,16:15,Celta Vigo,Villarreal,laliga
